In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE145701"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE145701"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE145701.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE145701.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE145701.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Steroid Hormones Regulate Genome-Wide Epigenetic Programming and Gene Transcription in Human Endometrial Cells with Marked Aberrancies in Endometriosis [expression]"
!Series_summary	"Gene expression profiling of hormone treated normal and endometriosis stromal fibroblast cells (eSF). We used Affymetrix Human Gene 1.0 ST arrays. Samples include 4 normal of no uterine pathology (NUP), 4 endometriosis stage I, 4 endometriosis stage IV samples, each treated with Estrogen (E2), Progesterone (P4), E2+P4, or vehicle (veh), for a total of 48 samples on the Affymetrix platform."
!Series_overall_design	"eSF from eutopic endometrial biopsies from women without any pelvic or uterine complications as controls and from women with stage I or stage IV endometriosis were grown in culture and treated with ovarian steroid hormone for 14 days in culture. After treatment, cells were harvested and were tested for changes in gene expression (as well as DNA methylation) 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
import json
from typing import Optional, Callable, Dict, Any
import re

# 1. Analyze if gene expression data is available
# From the background information, we can see that this dataset contains gene expression data
# using Affymetrix Human Gene 1.0 ST arrays
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we can identify:
# - trait data is in row 2 (disease state)
# - age data is not available
# - gender is in row 0, but it's constant (all Female)

trait_row = 2  # disease state: Normal/Endometriosis Stage I/Endometriosis Stage IV
age_row = None  # Age information is not available
gender_row = None  # Gender is constant (all Female), so it's not useful for our study

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert disease state to binary (0: Normal, 1: Endometriosis)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'Normal' in value:
        return 0
    elif 'Endometriosis' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to continuous value (not used in this dataset)"""
    return None  # Not available in this dataset

def convert_gender(value):
    """Convert gender to binary (0: Female, 1: Male) (not used in this dataset)"""
    return None  # Constant value (all Female) in this dataset

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    sample_chars = {
        0: ['gender: Female'], 
        1: ['cell type: Eutopic endometrial stromal fibroblasts'], 
        2: ['disease state: Normal', 'disease state: Endometriosis Stage I', 'disease state: Endometriosis Stage IV'], 
        3: ['treatment: E2', 'treatment: P4', 'treatment: E2+P4', 'treatment: vehicle'], 
        4: ['individual: NUP1', 'individual: NUP2', 'individual: NUP3', 'individual: NUP4', 'individual: Endo I-1', 
            'individual: Endo I-2', 'individual: Endo I-3', 'individual: Endo I-4', 'individual: Endo IV-1', 
            'individual: Endo IV-2', 'individual: Endo IV-3', 'individual: Endo IV-4']
    }
    
    # Convert the dictionary to a DataFrame that can be used with geo_select_clinical_features
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of the selected clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of the selected clinical data:
{0: [0.0], 1: [1.0], 2: [1.0], 3: [nan], 4: [nan], 5: [nan], 6: [nan], 7: [nan], 8: [nan], 9: [nan], 10: [nan], 11: [nan]}
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE145701.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 71
Header line: "ID_REF"	"GSM4331199"	"GSM4331200"	"GSM4331201"	"GSM4331202"	"GSM4331203"	"GSM4331204"	"GSM4331205"	"GSM4331206"	"GSM4331207"	"GSM4331208"	"GSM4331209"	"GSM4331210"	"GSM4331211"	"GSM4331212"	"GSM4331213"	"GSM4331214"	"GSM4331215"	"GSM4331216"	"GSM4331217"	"GSM4331218"	"GSM4331219"	"GSM4331220"	"GSM4331221"	"GSM4331222"	"GSM4331223"	"GSM4331224"	"GSM4331225"	"GSM4331226"	"GSM4331227"	"GSM4331228"	"GSM4331229"	"GSM4331230"	"GSM4331231"	"GSM4331232"	"GSM4331233"	"GSM4331234"	"GSM4331235"	"GSM4331236"	"GSM4331237"	"GSM4331238"	"GSM4331239"	"GSM4331240"	"GSM4331241"	"GSM4331242"	"GSM4331243"	"GSM4331244"	"GSM4331245"	"GSM4331246"
First data line: 7892501	5.71675	3.834201	4.159606	5.485716	6.177502	3.268134	5.447216	4.924031	6.030231	5.966432	5.049547	5.640725	5.178561	5.42022	5.878601	5.105815	5.226443	5.879782	5.619013	5.150586	5.291864	4.175157	6.070517	4.793396	6.301941	3.389179	6.073273	4.432051	4.978615	5.589209	5.530389	5.291379	4.393631	4.501

### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers in the gene expression data from the output above,
# we see numeric identifiers like '7892501', '7892502', etc.
# These appear to be Affymetrix probe IDs rather than standard human gene symbols.
# Human gene symbols would typically be alphanumeric like 'BRCA1', 'TP53', etc.
# Therefore, we will need to map these probe IDs to proper gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. First, identify which columns contain probe IDs and gene symbols
# From the gene annotation preview, 'ID' column contains probe IDs like the ones in gene_data
# The 'gene_assignment' column contains information about gene symbols

# Get the gene mapping dataframe with probe ID and gene symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 2. Preview the mapping data to check its structure
print("Gene mapping preview (first 5 rows):")
print(preview_df(mapping_df.head(5)))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the gene expression data
print("\nGene expression data preview (first 5 genes):")
print(preview_df(gene_data.head(5)))

# Check the dimensions of the gene expression data
print(f"\nGene expression data dimensions: {gene_data.shape}")

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'Gene': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST00000492842 // OR4G11P // olfactory receptor, family 4, subfamily G, member 11 pseudogene // --- // --- /// ENST00000588632 // OR4G1P // olfactory receptor, family 4, subfamily G, member 1 pseudogene // --- // ---', 'NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001005484 // OR4F5 // olfactory receptor, family 4, subfamily F, member 5 // 1p36.33 // 79501 /// ENST00000318050 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// ENST00000326183 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// ENST00000335137 // OR4F5 // olfactor


Gene expression data preview (first 5 genes):
{'GSM4331199': [27.47600297161529, 0.6446407000000001, 1.5151557142857144, 1.0442910133333334, 1.9833253232323234], 'GSM4331200': [27.551133794557426, 0.6341504, 1.5079428571428573, 0.9933030608695652, 1.975007424242424], 'GSM4331201': [27.838618534898433, 0.6955715, 1.500627142857143, 1.0127446140786749, 1.9672003333333332], 'GSM4331202': [28.155517359870686, 0.62011675, 1.4976657142857144, 1.0558890285507245, 1.965546404040404], 'GSM4331203': [26.874872071496362, 0.5827172, 1.5269957142857142, 1.0150620322567288, 1.9387945858585858], 'GSM4331204': [29.123107917054373, 0.6827809499999999, 1.5167957142857145, 1.0559544466252588, 2.0392911616161618], 'GSM4331205': [27.54240771930411, 0.6599256499999999, 1.5270899999999998, 1.065540886293996, 1.97742498989899], 'GSM4331206': [28.407915610743622, 0.6484231, 1.536072857142857, 1.0474513733954451, 2.0517297777777777], 'GSM4331207': [27.569580890637535, 0.63460765, 1.5441257142857143, 0.96665169

Gene expression data saved to ../../output/preprocess/Endometriosis/gene_data/GSE145701.csv


### Step 7: Data Normalization and Linking

### Step 8: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE145701.csv
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE145701.csv
Linked data shape: (60, 24230)
Quartiles for 'Endometriosis':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Endometriosis' in this dataset is fine.

Abnormality detected in the cohort: GSE145701. Preprocessing failed.
Data was determined to be unusable and was not saved
